In [5]:
import cv2 as cv
import pytesseract
pytesseract.pytesseract.tessaract_cmd = r'/usr/local/Cellar/tesseract/4.1.1/bin/tesseract'
import nltk
import os 
import re
import matplotlib.pyplot as plt
from skimage import data
from skimage.color import rgb2gray
from PIL import Image
import PIL.ImageOps 
os.chdir('/Users/jpjansen/Desktop/5_9_21/images')
import numpy as np
import glob
import xlsxwriter
from pytesseract import Output

In [6]:
print(os.getcwd())


/Users/jpjansen/Desktop/5_9_21/images


In [4]:
#save as grayscale
for filename in os.listdir(os.getcwd()):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        #image = Image.open(filename)
        #inverted_image = ImageOps.invert(image)
        img = Image.open(filename).convert('LA')
        export = filename.split(".")
        img.save(export[0]+'gray.png')

In [7]:

image_one = 0
image_two = 0 
image_three = 0
image_four = 0
image_one_data = [] 
image_three_data = [] 
inflows = []
outflows = [] 
image_two_data = []
image_threes = [] 
image_four_data = [] 
stock = ['XSPA', 'RKDA', 'ETH', 'RHE', 'CCXI', 'BBIG', 'OBLN', 'APOP', 'PRFX', 'YTEN', 'BGFV', 'VISL', 'CDEV', 'ANY', 'FTOC', 'CLOV', 'PRPO', 'BTX', 'BCRX', 'MGI', 'NKLA', 'LEDS', 'PLTR', 'MVIS', 'OCGN', 'CREG', 'SEAC', 'ADVM', 'SONN', 'CCIV']
free_float = [] 
picture_detection = [] 
noise_list = [',','.',r'/', '(', ')']
market_names = ['NASDAQ','NASDA','NASD', 'NAS', 'NA','NYSE', 'NYS','NY','NYSEARCA' ,'AMEX', 'AME', 'AM']
undetermined_names = []

for filename in sorted(os.listdir(os.getcwd())):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        print(filename) 
        img = cv.imread(filename)
        #img =  rgb2gray(img)
        ret,thresh1 = cv.threshold(img,241,255,cv.THRESH_BINARY)
        #kernel = np.ones((5,5), np.uint8)
        #img_dilation = cv.dilate(thresh1, kernel, iterations=10) 
        #plt.imshow(img)
        #ret,thresh1 = cv2.threshold(img,0,255,cv2.THRESH_OTSU)
        #plt.imshow(img)
        
        #--psm 12 -c tessedit_char_whitelist= 0123456789
        text = pytesseract.image_to_string(thresh1,config='--psm 6')
        #print(text)
        nltk_tokens = nltk.word_tokenize(text)
        get_stock_name=[]
        for i in range(len(nltk_tokens)):
            if nltk_tokens[i] not in noise_list:
                get_stock_name.append(nltk_tokens[i])
        print(nltk_tokens)
        #cv.imshow('picture',thresh1)
        #cv.waitKey(0)
        print('FILENAME',filename)
        print(get_stock_name)
        #print(nltk_tokens)
        for i in range(len(get_stock_name)):
            if get_stock_name[i] in market_names:
                if get_stock_name[i-1] in stock:
                    name = get_stock_name[i-1]
                    print(name)
                break
            else:
                name = 'NaN'
                print('Couldnt determine name')
        if name=='NaN':
            undetermined_names.append(filename)
                       
        for i in range(len(nltk_tokens)):
            multiplier = 1 
            #if nltk_tokens[i] in stock:
                #name = nltk_tokens[i]
            if nltk_tokens[i]=="NEXT" and nltk_tokens[i+1]=="EARNINGS":
                print("image 4 detected")
                #print(nltk_tokens)
            if nltk_tokens[i]=="EX-DIV" and nltk_tokens[i+1]=='DATE':
                free_float_value = nltk_tokens[i+2]
                if free_float_value=="--":
                    free_float.append((name,'NaN'))
                    break
                print(free_float_value)
                if 'K' in free_float_value:
                    free_float_value = free_float_value.replace("K","")
                    multiplier = .001
                elif 'M' in free_float_value:
                    multiplier = 1
                    free_float_value = free_float_value.replace("M","")
                elif 'B' in free_float_value:
                    multiplier = 1000
                    free_float_value = free_float_value.replace("B","")
                elif 'T' in free_float_value:
                    multiplier = 1000000
                    free_float_value = free_float_value.replace("T","")
                if "." not in free_float_value and "," in free_float_value:
                    commas = []
                    for i in range(len(free_float_value)):
                        if free_float_value[i]==",":
                            commas.append(i)
                    replace = np.max(commas)
                    new_str = list(free_float_value)
                    del(new_str[replace])
                    new_str.insert(replace,".")
                    final = ''.join(new_str)
                    final.replace(",","")
                    free_float_value = final
                elif "." and "," in free_float_value:
                    continue
                if free_float_value != 'NaN':
                    free_float.append((name,float(free_float_value)*multiplier))
                multiplier = 1 
                
                picture_detection.append((name,filename,4))       
                image_four+=1
                #for i in range(len(nltk_tokens)):
                    #if nltk_tokens[i] == 
                
                image_four_data.append(nltk_tokens)
                picture_detection.append((name,filename,4))
                break
            if nltk_tokens[i]=="Order" and nltk_tokens[i+1]=="Flow" and nltk_tokens[i+2]=="Distribution":
                print("Image 2 detectd")
                multiplier = 1
                inflow = 'NaN'
                outflow = 'NaN'
                picture_detection.append((name,filename,2))
                for i in range(len(nltk_tokens)):
                    if nltk_tokens[i]=='Inflow':
                        inflow = nltk_tokens[i+2].replace(",","")
                        print('Inflow',inflow)
                    if nltk_tokens[i]=='Outflow':
                        outflow = nltk_tokens[i+2].replace(",","")
                        print('Outflow', outflow)
                for i in range(len(nltk_tokens)):
                    if nltk_tokens[i]=='Million' and nltk_tokens[i+1] =='USD':
                        multiplier = 1000000
                        break
                    if nltk_tokens[i]=='Kilo' and nltk_tokens[i+1]=='USD':
                        multiplier = 1000 
                        break
                image_two+=1
                inflows.append((name,float(inflow)*multiplier))
                outflows.append(float(outflow)*multiplier)
                #image_two_data.append((name,float(inflow*multiplier),float(outflow*multiplier)))
                multiplier = 1 
                break 
            if nltk_tokens[i]=="Large" and nltk_tokens[i+1]=="Scale" and nltk_tokens[i+2]=="Orders" and nltk_tokens[i+3] == "in" and nltk_tokens[i+4] == "Last"and nltk_tokens[i+5] == "5"and nltk_tokens[i+6] == "Days":
                print("image 3 detected")
                picture_detection.append((name,filename,3))
                img_3_export = filename.split(".")
                exp_image3 = Image.open(filename)
                export = filename.split(".")
                exp_image3.save('/Users/jpjansen/Desktop/5_9_21/image threes/'+export[0]+'image3.png')
                #print(nltk_tokens)
                multiplier = 1 
                if nltk_tokens[i+7]=="Kilo":
                    large_multiplier = 1000
                if nltk_tokens[i+7]=="Million":
                    large_multiplier = 1000000
                if nltk_tokens[i+7]=="USD": 
                    large_multiplier = 1
                image_threes.append((name,filename,large_multiplier))
                image_three+=1
                break
          
            if nltk_tokens[i]=='Order' and nltk_tokens[i+1]=='Book' and nltk_tokens[i+2]=='-'and nltk_tokens[i+3]=='Level':
                # old img one detection"AMA" and "DMA" in nltk_tokens:
                print("image 1 detected")
                #print(nltk_tokens)
                image_one+=1
                #image_one_data.append(nltk_tokens)
                #for i in range(len(nltk_tokens)):
                    #if nltk_tokens[i]=='NASDAQ'or nltk_tokens[i]=='NYSE'or nltk_tokens[i]=='AMEX':
                        #name = nltk_tokens[i-1]
                        #break
                #img = cv.imread(filename)
                #ret,thresh1 = cv.threshold(img,235,255,cv.THRESH_BINARY)
                #kernel = np.ones((5,5), np.uint8)
                #img_dilation = cv.dilate(thresh1, kernel, iterations=10)
                #cv.imshow('picture',thresh1)
                #cv.waitKey(0)
                #text = pytesseract.image_to_string(thresh1,config='--psm 6')
                #nltk_tokens = nltk.word_tokenize(text)
                for i in range(len(nltk_tokens)):
                    if re.match(r"MA10:",nltk_tokens[i])!=None:
                        ten_day_ma = nltk_tokens[i]
                        if "." not in ten_day_ma:
                            try:
                                commas = []
                                for i in range(len(ten_day_ma)):
                                    if ten_day_ma[i]==",":
                                        commas.append(i)
                                replace = np.max(commas)
                                new_str = list(ten_day_ma)
                                del(new_str[replace])
                                new_str.insert(replace,".")
                                final = ''.join(new_str)
                                final.replace(",","")
                                ten_day_ma = float(final[1])
                                
                                break
                            except:
                                print('VALUE ERROR')
                                ten_day_ma = 'NaN'
                                break
                        else:
                            try:
                                ten_day_ma.replace(",","")
                                ten_day_ma = ten_day_ma.split(":")
                                ten_day_ma = float(ten_day_ma[1])
                                break
                            except ValueError: 
                                ten_day_ma= 'NaN'
                                break 
                print(ten_day_ma)
                #ten_day_ma = ten_day_ma.split(":")
                #print(ten_day_ma)
                picture_detection.append((name,filename,1))
                image_one_data.append((name,ten_day_ma))
                break
            
print(image_one,image_two,image_three,image_four)
#print("Image One data:", image_one_data)
#print("Image Two data:", image_two_data)
#print("Image Three data:", image_three_data)
print("Image Four data:",image_four_data)
print(free_float)
print(inflows)
print(outflows)
print(image_one_data)
print(np.asarray(picture_detection))

IMG_8284gray.jpeg
['10:32', '7', 'all', '>', '<', '4', 'Search', '<', 'XSPA', 'NASDAQ', 'YY', 'XPRESSPA', 'GROUP', '*', '+0.020', '1', '.260', '+1.61', '%', 'BB', '-0.110', '(', '-8,03', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '1.250/1.240', '1.231-1.330', '2.22M', '1D', '5D', '1M', '3M', '1Y', '5Y—', 'Max', '|A', '88', 'MAS:1.258', 'MA10:1.264', 'MA20:1.265', '1.398', '‘', '_', '-', '#', '2.01', '%', 'ee', 'N', 'hi', '|', '|', '“', '1.66', '%', 'Nhe', '’', 'Ay', 'ie', 'Lay', '|', 'nA', 'i', '>', 'Bye', '3', '%', 'wee', 'eee', 'eee', 'eee', 'eee', 'ee', 'PR', 'an', '_', 'ih', 'me', 'a', 'A', ',', '2', '1.247', 'A', 'Hh', 'Li', '\\\\', '|', 'es', 'yO', 'DY', 'LL', 'iy', ',', 'a', ':', ':', ':', '_', '@', '-', '—', 'dab', ',', 'a', 'atuue', 'd', '—', 'thc', 'oe', 'alenwaladball', 'had', 'Least', 'Lal', 'L', 'WM', 'ie', 'aia', 'lal', '05/03', '05/04', '05/05', '05/06', '05/07', '05/08', 'DMA', '(', '10,50,10', ')', 'DDD', ':', '-0.002', 'AMA', ':', '-

['10:32', '9', 'al', 'F', '©', '<', '4', 'Search', 'RKDA', 'NA', '<', 'SDAQ', 'vy', 'Arcadia', '+0.120', '2.600', '+4.84', '%', 'SB', '-0.110', '(', '-4.14', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '2.480/2.480', '2.480-2.600', '609.37K', '1D', '5D', '1M', '3M', '1Y-', '5Y—', 'Max', '|A-', '88', 'MAS:2.558', 'MA10:2.565', 'MA20:2.569', ':', '2.776', ';', '44.34', '%', '|', '3', '3', '.', '682', 'Co', '+0.84', '%', 'I', '--', '--', '--', '--', '--', '-', '=e', ',', ':', 'ea', 'eh', 'hon', 'ie', 'in', 'rh', '_', 'Wr', 'promo', 'oe', 'RIN', '~~~', 'aa', '|', 'VI', 'f', 'WF', '+e', '0', 'A', 'sp', '{', '\\', 'yc', ':', "'", '|', 'hee', 'Fy', ';', '1A', 'IN', '=', ':', ':', 've', '|', '-', 'Tides', 'sw', 'deb', '!', 'wall', 'vol', 'ach', 'uk', 'Jal', 'albvond', '‘', 'ok', 'sissbsldbinseerlblan', '05/03', '05/04', '05/05', '05/06', '05/07', 'DMA', '(', '10,50,10', ')', 'DDD:0.014', 'AMA', ';', '0,024', '<', '0:04', 'f', ',', '62', 'e', 'Quotes', 'News', 

['10:32', '7', 'al', 'FO', '<', '4', 'Search', 'ETH', 'NYSE', '<', 'Ethan', 'Allen', 'Int', 'w', '+0.89', '31', '.99', '+2.86', '%', 'SEB', '+3,39', '(', '+11.81', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '31.57/31.10', '30.98-32.10', '824.56K', '@', 'ETH', 'reported', 'a', 'USD', '0.25/share', 'interim', 'dividend', ',', 'ex-dividendo', 'X', '1D', '5D', '1M', '3M', '1Y', 'SY', 'Max', '|A-', '88', 'MA5:32.03', 'MAI', '0:32.04', 'MA20:32.01', '|', '3206______.___-', '88', 'LEZ', '|', '31.26', 'j', ':', '|', 'Ay', 'Pr', '.', '``', '\\', '|', 'Ne', 'Air', 'oo', 'see', 'D', 'WV', 'Nets', 'gecmEN', '77', ',', '7', '30.47', 'ae', 'pee', '3', 'b', 'V/', 'V', ':', 'ee', '...', 'acasinabt', 'Menai', 'dtc', 'rac', 'eae', 'el', '05/03', '05/04', '05/05', '05/06', '05/07', 'DMA', '(', '10,50,10', ')', 'DDD:0.15', 'AMA:0.17', '—_', '0:27', "'", '-0.02', 'e', 'e', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Order', 'Book', '-', 'Level', '1', '

IMG_8295gray.jpeg
['10:32', '9', 'all', 'F', '<', '4', 'Search', 'ETH', 'NYSE', '<', 'Ethan', 'Allen', 'Int', 'Ww', '+0.89', '31', '99', '+2.86', '%', 'Bean', '+3,39', '(', '+11.81', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '31.57/31.10', '30.98-32.10', '824.56K', 'MARKET', 'CAP', '52', 'WEEK', 'RANGE', 'AVG', 'VOL', '(', '3M', ')', '806.05M', '9.08-32.10', '351.92K', '%', 'TURNOVER', 'TURNOVER', '@', 'P/E', '(', 'TTM', ')', '3.27', '%', '-', '27.27', '%', 'RANGE', 'EPS', '(', 'TTM', ')', 'P/E', '(', 'FWD', ')', '3.62', '%', '1.173', '15.05', 'P/B', 'BVPS', 'DIVIDEND', '2.260', '14.16', '1.000', 'P/S', 'BETA', 'DIV', 'YIELD', '1.351', '-', '3.22', '%', 'FREE', 'FLOAT', 'MKT', 'CAP', 'SHARES', 'OUTSTANDING', 'NEXT', 'EARNINGS', '684.23M', '25.2M', '08/02-08/06', 'FREE', 'FLOAT', 'LOT', 'SIZE', 'EX-DIV', 'DATE', '21.39M', '1', '05/10/2021', '@', 'im', 'dividend', ',', 'ex-dividend', 'on', 'Mon', 'May', '10', ',', '2021', 'ET', '.', 'E', 'xX', '1D', '5

IMG_8299gray.jpeg
['10:33', '7', 'ule', '<', '4', 'Search', 'RHE', 'AMEX', '<', 'Ww', 'Regional', 'Health', '+3.62', '18.60', '+24.17', '%', 'SB', '+10.79', '(', '+149.45', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '16.65/14.98', '16.50-23.80', '54.87M', 'MARKET', 'CAP', '52', 'WEEK', 'RANGE', 'AVG', 'VOL', '(', '3M', ')', '31.40M', '1.120-27.72', '4.08M', '%', 'TURNOVER', 'TURNOVER', '@', 'P/E', '(', 'TTM', ')', '3,243.10', '%', '-', 'N/A', '%', 'RANGE', 'EPS', '(', 'TTM', ')', 'P/E', '(', 'FWD', ')', '48.66', '%', '-5.6878', 'N/A', 'P/B', 'BVPS', 'DIVIDEND', '2.828', '6.58', '--', 'P/S', 'BETA', 'DIV', 'YIELD', '1.786', '-', '0.00', '%', 'FREE', 'FLOAT', 'MKT', 'CAP', 'SHARES', 'OUTSTANDING', '—', 'NEXT', 'EARNINGS', '28.08M', '1.69M', '--', 'FREE', 'FLOAT', 'LOT', 'SIZE', 'EX-DIV', 'DATE', '1.51M', '1', '01/02/2019', '1D', '5D', '1M', '3M', '1Y', 'SY', 'Max', '|A-', 'BS', 'MAS:18.04', 'MA10:18.04', 'MA20:17.96', ':', '26.28', '(', '-', '#', '264,0

...

IMG_8383gray.jpeg
['10:38', '7', 'ul', '>', '@', '&', ')', '<', 'OCGN', 'NASDAQ', 'iy', 'Ocugen', 'Inc', '-0.40', '9', 'e', '1', '9', '-4.12', '%', 'B', '-3,61', '(', '-28,49', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '8.04/9.58', '7.61-9.86', '177.79M', 'MARKET', 'CAP', '52', 'WEEK', 'RANGE', 'AVG', 'VOL', '(', '3M', ')', '1.82B', '0.1700-18.77', '75.52M', '%', 'TURNOVER', 'TURNOVER', '@', 'P/E', '(', 'TTM', ')', '89.38', '%', '-', 'N/A', '%', 'RANGE', 'EPS', '(', 'TTM', ')', 'P/E', '(', 'FWD', ')', '23.49', '%', '-0.3646', '4.185', 'P/B', 'BVPS', 'DIVIDEND', '78.44', '0.1171', '--', 'P/S', 'BETA', 'DIV', 'YIELD', '24171.05', '-', '0.00', '%', 'FREE', 'FLOAT', 'MKT', 'CAP', 'SHARES', 'OUTSTANDING', '—', 'NEXT', 'EARNINGS', '1.77B', '198.17M', '--', 'FREE', 'FLOAT', 'LOT', 'SIZE', 'EX-DIV', 'DATE', '193.21M', '1', '09/30/2019', '1D', '5D', '1M', '3M', '1Y', 'SY', 'Max', '|A-', 'BS', 'MAS:9.11', 'MA10:9.13', 'MA20:9.11', '|', ':', '16.04', '+26.61', 

IMG_8387gray.jpeg
['10:38', '7', 'all', 'Ff', '@', '&', ')', 'CREG', 'NASDAQ', '<', 'CREG', 'Ww', '+0.60', '9', '°', '0', '9', '+7.07', '%', 'SB', '-0.47', '(', '-5.05', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '8.99/8.49', '8.53-9.90', '495.06K', 'MARKET', 'CAP', '52', 'WEEK', 'RANGE', 'AVG', 'VOL', '(', '3M', ')', '28.88M', '2.100-14.39', '196.6K', '%', 'TURNOVER', 'TURNOVER', '@', 'P/E', '(', 'TTM', ')', '15.58', '%', '-', '7.06', '%', 'RANGE', 'EPS', '(', 'TTM', ')', 'P/E', '(', 'FWD', ')', '16.14', '%', '1.288', '18.86', 'P/B', 'BVPS', 'DIVIDEND', '0.3134', '29.01', '--', 'P/S', 'BETA', 'DIV', 'YIELD', '-3329.6703', '-', '0.00', '%', 'FREE', 'FLOAT', 'MKT', 'CAP', 'SHARES', 'OUTSTANDING', '—', 'NEXT', 'EARNINGS', '24.43M', '3.18M', '-', 'FREE', 'FLOAT', 'LOT', 'SIZE', 'EX-DIV', 'DATE', '2.69M', '100', '04/15/2020', '1D', '5D', '1M', '3M', '1Y', 'SY', 'Max', '|A-', 'BS', 'MAS:9.01', 'MA10:9.01', 'MA20:8.93', '!', ':', ':', '9,60', ':', '43.26', 

['10:38', '7', 'all', '>', '@', ')', 'ADVM', 'NA', '<', 'SDAQ', 'wy', 'Adverum', '+0.090', '3.780', '+2.44', '%', 'SB', '-0.120', '(', '-3.08', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '3.640/3.690', '3.600-3.870', '3.35M', '1D', '5D', '1M', '3M', '1Y', '5Y—', 'Max', '|A', '88', 'MAS:3.764', 'MA10:3.763', 'MA20:3.769', '(', '(', '4.231', '+8.48', '%', '*h', '|', '|', '|', '|', 'ay', '2', ':', 'sim', '384MR', '8', '86', "''", '|', 'i', 'rn', '/', 'P', 'iy', 'y', 'iy', 'age', 'y', 'ry', '3', 'ad', 'ue', 'toMy', 'aS', 'eee', 'ft', 'ee', '|', 'ae', '|', 'en', '|', '_', 'ean', '|i', 'vee', 'Lo', 'whit', 'teil', '|', '05/03', '05/04', '05/05', '05/06', '05/07', '«', '05/08', 'DMA', '(', '10,50,10', ')', 'DDD', ':', '-0,029', 'AMA', ':', '-0.035', '0:07', 'i', 'wi', '-0', 'e', 'e', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Order', 'Book', '-', 'Level', '1', '338', 'Trade', '&', 'e', 'eee', 'Positions', 'Options', 'Comment', 'More', 'e

['10:38', '7', 'all', '>', '@', ')', '<', 'SONN', 'NASDAQ', 'YY', 'Sonnet', 'BioTherapeutics', 'Inc', '+', '-0.070', '1', '.620', '4.14', '%', 'SB', '-0.690', '(', '-29.11', '%', ')', 'Last', '1W', 'OPEN/PREV', 'CLOSE', 'DAY', "'S", 'RANGE', 'VOLUME', '1.716/1.690', '1.610-1.720', '1.31M', '1D', '5D', '1M', '3M', '1Y', '5Y—', 'Max', '|A', '88', 'MAS5:1.648', 'MA10:1,644', "MA20:1.636'", '2.656', ':', ':', '+12.08', '%', 'i', 'i', 'nal', ':', ':', 'yh', 'Mh', ',', ';', ':', '11.71', '%', '1,810', 'ne', '»', 'VO', 'we', '|', '+23.61', '%', '05/03', '05/04', '05/05', '05/06', '05/07', '05/08', 'DMA', '(', '10,50,10', ')', 'DDD:0.010', 'AMA:0.001', '<', '0:03', '-0:10', 'e', 'e', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Order', 'Book', '-', 'Level', '1', '338', 'Trade', '&', 'e', 'eee', 'Positions', 'Options', 'Comment', 'More', 'ee']
FILENAME IMG_8396gray.jpeg
['10:38', '7', 'all', '>', '@', '<', 'SONN', 'NASDAQ', 'YY', 'Sonnet', 'BioTherapeutics', 'Inc', '+', '-0.0

['<', '19.28', '40.32/F1.69', '%', 'Ww', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Million', 'USD', '1,28', '%', '38.36', '%', '|', '36.09', '%', 'Orders', '12.69', '%', 'SZ', '...', '46', '43', '15', '14', '0', '2', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '61', 'Outflow', ':', '58', 'Large', 'Scale', 'Orders', 'in', 'Last', '5', 'Days', 'Kilo', 'USD', '1,878', '388', '0', '-1,038', '-1,530', 'CCIV-U', '(', 'NYSE', ')', '20.98', '+0.43', '+2.11', '%', 'Positions', 'Options', 'Comment', 'More', 'ee']
FILENAME IMG_8401gray.jpeg
['<', '19.28', '40.32/F1.69', '%', 'Ww', 'Quotes', 'News', 'Comments', 'Analysis', 'Press', 'Releases', 'Order', 'Flow', 'Distribution', '©', 'Million', 'USD', '1,28', '%', '38.36', '%', '|', '36.09', '%', 'Orders', '12.69', '%', 'SZ', '...', '46', '43', '15', '14', '0', '2', 'Large', 'Medium', 'Small', 'Large', 'Medium', 'Small', 'Inflow', ':', '61', 'Outflow', ':', '58', '

In [8]:
os.chdir('/Users/jpjansen/Desktop/5_9_21/image threes')

large_scale_buying = [] 
counter = 0 
image_threes = np.asarray(image_threes)
image_three_resave = []
for filename in sorted(os.listdir(os.getcwd())):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        if filename == '.DS_Store':
            continue
        print(filename)
        stock_name = image_threes[counter,0]
        final_multiplier = image_threes[counter,2]
        image_three_resave.append(filename)
        counter+=1
        #,cv.IMREAD_GRAYSCALE
        img = cv.imread(filename)
        height = img.shape[0]
        print('height', height)
        ret,thresh1 = cv.threshold(img,240,255,cv.THRESH_BINARY)
        height_2 = thresh1.shape[0]
        print('Height 2', height_2)
        kernel = np.ones((2,2), np.uint8)
        img_dilation = cv.erode(thresh1, kernel, iterations=3) 
        boxes = pytesseract.image_to_data(thresh1,output_type= Output.DICT,config='--psm 6 --oem 3 -c tessedit_char_whitelist=/.,-0123456789')
        good_words = [] 
        good_top = [] 
        remove = ['/',',','.','','-', None]
        for z in range(len(boxes['text'])):
            #print(len(boxes['text'][z]))
            if len(boxes['text'][z])==0:
                #print('Zero')
                continue
            if boxes['text'][z] not in remove:
                good_words.append(boxes['text'][z])
                good_top.append(boxes['top'][z])
        #print('good words', good_words)
        #print('good top', good_top)
        buy_locations = []
        buy_values = []
        for i in range(len(good_words)):
            #print(i, nltk_tokens[i])
                if re.match(r"[0-9][0-9]/[0-9][0-9]",good_words[i])!=None:
                    if re.match(r"[0-9][0-9]/[0-9][0-9]",good_words[i+1])!=None:
                        if good_words[i-6]=='5':
                            print(filename)
                            print(good_words)
                            print(good_top[i]/height)
                            date_dist = good_top[i]/height
                            print('MATCH')
                            final_multiplier = float(final_multiplier)
                            print('buy 1')
                            buy1 = good_words[i-1].replace(",","")
                            print('buy 1',buy1)
                            buy1_loc = (good_top[i-1]/height)-date_dist
                            print('buy 1 location', buy1_loc)
                            buy2_loc = (good_top[i-2]/height)-date_dist
                            buy2 = good_words[i-2].replace(",","")
                            print('buy 2',buy2)
                            print('buy 2 location', buy2_loc)
                            buy3 = good_words[i-3].replace(",","")
                            print('buy 3',buy3)
                            buy3_loc = (good_top[i-3]/height)-date_dist
                            print('buy 3 location', buy3_loc)
                            buy4 = good_words[i-4].replace(",","")
                            print('buy 4',buy4)
                            buy4_loc = (good_top[i-4]/height)-date_dist
                            print('buy 4 location', (good_top[i-4]/height)-date_dist)
                            buy5 = good_words[i-5].replace(",","")
                            print('buy 5',buy5)
                            buy5_loc = (good_top[i-5]/height)-date_dist
                            print('buy 5 locaion', (good_top[i-5]/height)-date_dist)
                            buy_locations.append(buy1_loc)
                            buy_locations.append(buy2_loc)
                            buy_locations.append(buy3_loc)
                            buy_locations.append(buy4_loc)
                            buy_locations.append(buy5_loc)
                            buy_values.append(buy1)
                            buy_values.append(buy2)
                            buy_values.append(buy3)
                            buy_values.append(buy4)
                            buy_values.append(buy5)
                            break
                        else:
                            print('More than 5 values')
                            print(good_words)
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_locations.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            buy_values.append('Five val problem')
                            break
        buy_values_floats = [] 
        for i in range(len(buy_values)):
            try:
                buy_values_floats.append(float(buy_values[i])*final_multiplier)
            except ValueError:
                try:
                    check_7 = buy_values[i].replace("/","7")
                    buy_values_floats.append(float(check_7)*final_multiplier)
                except ValueError:
                    buy_values_floats.append('NaN')
                
        final_values = [] 
        zero_or_negative = 'NaN'
        zero_or_negative_matrix = []
        positive_check = []
        positive_check_loc = [] 
        ##check to see if the location of smallest positive number is higher than the location
        #of a larger positive number (would indicate negative)
        for i in range(len(buy_values_floats)):
            if type(buy_values_floats[i]) == str:
                continue
            if buy_values_floats[i]>0:
                positive_check.append(buy_values_floats[i])
                positive_check_loc.append(buy_locations[i])
        #get location of values detected as zero or negative          
        for i in range(len(buy_values_floats)):
            if type(buy_values_floats[i]) == str:
                continue
            if buy_values_floats[i]<=0:
                zero_or_negative_matrix.append(buy_locations[i])
                print('ZERO OR NEG', zero_or_negative_matrix)
            else:
                asdf = 1 
        for i in range(len(zero_or_negative_matrix)):
            if type(zero_or_negative_matrix[i])== float:
                zero_or_negative = 1
                break
            else:
                zero_or_negative = 'NaN'
                
        if len(positive_check_loc)>1:        
            positive_check_min_loc = positive_check_loc[np.argmin(positive_check)]
            pos_minimum = np.amin(positive_check)
            fx = 0 
            for i in range(len(positive_check)):
                value = positive_check[i]
                coord = positive_check_loc[i]
                for x in range(len(positive_check)):
                    if value> positive_check[x] and coord>positive_check_loc[x]:
                        print('coord higher than min')
                        zero_or_negative = 'Coord problem'
                        break
                    else:
                        continue
                            
        print(buy_values_floats)
        problem = False
        positive_values = []
        distance_ratios = [] 
        for i in range(len(buy_values_floats)):
            
            if zero_or_negative!= 'NaN' or 'Coord problem':
                if buy_values_floats[i] == 'NaN':
                    print('Final values append didnt read number well')
                    final_values.append('Check didnt read number well')
                    problem = True
                    continue
                pos_count1 = 0
                pos_count2= 0
                for x in range(len(zero_or_negative_matrix)): 
                    if buy_values_floats[i]<=0 and buy_locations[i]==zero_or_negative_matrix[x]:
                        final_values.append(buy_values_floats[i])
                        print('final values append zero or neg and zero or neg loc')
                        break
                    elif buy_values_floats[i]>0 and buy_locations[i]==zero_or_negative_matrix[x]:
                        final_values.append(buy_values_floats[i]*-1)
                        print('final val append pos but zero or neg loc')
                        break
                    elif buy_values_floats[i]<=0 and buy_locations[i]!=zero_or_negative_matrix[x]:
                        final_values.append(buy_values_floats[i])    
                        ('final val append neg but not zero or neg loc ')
                        break
                    elif buy_values_floats[i]>0 and buy_locations[i]!=zero_or_negative_matrix[x]:
                        if x==(len(zero_or_negative_matrix)-1):
                            print('positive value not equal to zero or neg')
                            final_values.append(buy_values_floats[i])
                            distance_ratios.append(buy_locations[i]/buy_values_floats[i])
                            break                                                                 
            if zero_or_negative=='Coord problem':
                final_values = [] 
                for i in range(len(buy_locations)):
                    print('final values append coord problem')
                    final_values.append('coord problem')
                    problem = True 
                break 
            elif zero_or_negative == 'NaN':
                if buy_values[1]== 'Five val problem':
                    for i in range(len(buy_locations)):
                        print('final values append five val problem')
                        final_values.append('Five val problem')
                        problem = True 
                        break 
                else:
                    (unique, counts) = np.unique(buy_locations, return_counts=True)
                    if np.max(counts)>1:
                        for i in range(len(buy_locations)):
                            print('final values appending multiple identical value locs')
                            final_values.append('Check multiple identical value locations')
                            problem = True
                        break
                    else:
                        for i in range(len(buy_locations)):
                            try:
                                final_values.append(buy_values_floats[i])
                                distance_ratios.append(buy_locations[i]/buy_values_floats[i])
                                print('All positive, double check')
                                print('final values appending all positive')
                                problem = True
                            except TypeError:
                                distance_ratios.append(0)
                                problem = True
                        break
                                    
                
        if len(positive_values)>1:               
            print('distance ratios', distance_ratios)
            for i in range(len(positive_check)):  
                print('abs val of Pos loc minus date dist',np.abs(positive_check_loc[i]))
                if np.abs(positive_check_loc[i])<.05 and distance_ratios[i]<-1e-7:
                    final_values = [] 
                    print(' final vals appending distance ratio problem!')
                    problem = True
                    for q in range(len(buy_values_floats)):
                        final_values.append('D.R. problem')
                    break 
        print("Problem", problem)
        if problem == True: 
            img_3_export = filename.split(".")
            exp_image3 = Image.open(filename)
            exp_image3.save('/Users/jpjansen/Desktop/Double Check/'+img_3_export[0]+'problem.png')
    print('BUY VALUES', final_values)
    large_scale_buying.append(final_values)
    if len(final_values)!= 5:
        print('APPENDING PROBLEM HERE')


    

IMG_8286grayimage3.jpeg
height 7596
Height 2 7596
IMG_8286grayimage3.jpeg
['1,260-40.0201.619', '2.82', '17.88', '29.69', '30.37', '9.21', '10.03', '408', '417', '245', '126', '137', '39', '573', '800', '5', '123', '-147', '-40', '-443', '-207', '05/03', '05/04', '05/05', '05/06', '05/07', '88']
0.84913112164297
MATCH
buy 1
buy 1 -207
buy 1 location -0.13796735123749349
buy 2 -443
buy 2 location -0.13796735123749349
buy 3 -40
buy 3 location -0.13796735123749349
buy 4 -147
buy 4 location -0.14178515007898895
buy 5 123
buy 5 locaion -0.16640337019483942
ZERO OR NEG [-0.13796735123749349]
ZERO OR NEG [-0.13796735123749349, -0.13796735123749349]
ZERO OR NEG [-0.13796735123749349, -0.13796735123749349, -0.13796735123749349]
ZERO OR NEG [-0.13796735123749349, -0.13796735123749349, -0.13796735123749349, -0.14178515007898895]
[-207000.0, -443000.0, -40000.0, -147000.0, 123000.0]
final values append zero or neg and zero or neg loc
final values append zero or neg and zero or neg loc
final values

height 7596
Height 2 7596
IMG_8318grayimage3.jpeg
['5', '2.73040.07042.634', '29.37', '39.17', '11.79', '2', '19.67', '175', '131', '88', '53', '0', '0', '184', '262', '5', '0', '-24', '0', '0', '0', '05/03', '05/04', '05/05', '05/06', '05/07', '33']
0.8509741969457609
MATCH
buy 1
buy 1 0
buy 1 location -0.17022116903633489
buy 2 0
buy 2 location -0.17022116903633489
buy 3 0
buy 3 location -0.17022116903633489
buy 4 -24
buy 4 location -0.17022116903633489
buy 5 0
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.17022116903633489, -0.17022116903633489]
ZERO OR NEG [-0.17022116903633489, -0.17022116903633489, -0.17022116903633489, -0.17022116903633489, -0.17022116903633489]
[0.0, 0.0, 0.0, -24000.0, 0.0]
final values append zero or neg and zero or neg loc
final values append

height 7596
Height 2 7596
IMG_8350grayimage3.jpeg
['1036-7', '3.840-0.660-14.67', '5.50', '7.70', '29.09', '37.89', '8.32', '11.50', '6,883', '5,285', '1,513', '2,083', '1,000', '1,400', '7,797', '10,366', '5', '5', '2', '-11', '0.07', '0', '05/03', '05/04', '05/05', '05/06', '05/07', '33']
0.8474196945760927
MATCH
buy 1
buy 1 0
buy 1 location -0.12914691943127965
buy 2 0.07
buy 2 location -0.1299368088467614
buy 3 -11
buy 3 location -0.12914691943127965
buy 4 2
buy 4 location -0.14573459715639814
buy 5 5
buy 5 locaion -0.17022116903633489
ZERO OR NEG [-0.12914691943127965]
ZERO OR NEG [-0.12914691943127965, -0.12914691943127965]
[0.0, 70000.0, -11000000.0, 2000000.0, 5000000.0]
final values append zero or neg and zero or neg loc
positive value not equal to zero or neg
final values append zero or neg and zero or neg loc
positive value not equal to zero or neg
positive value not equal to zero or neg
Problem False
BUY VALUES [0.0, 70000.0, -11000000.0, 2000000.0, 5000000.0]
IMG_8354grayi

height 7596
Height 2 7596
More than 5 values
['10387', '9.19--0.40-4.12', '0.17', '35.17', '39.97', '11.86', '12.76', '294', '259', '94', '87', '1', '1', '389', '346', '5', '705', '141', '-906', '-119', '0', '7', '05/03', '05/04', '05/05', '05/06', '05/07', '88']
['NaN', 'NaN', 'NaN', 'NaN', 'NaN']
Final values append didnt read number well
Final values append didnt read number well
Final values append didnt read number well
Final values append didnt read number well
Final values append didnt read number well
Problem True
BUY VALUES ['Check didnt read number well', 'Check didnt read number well', 'Check didnt read number well', 'Check didnt read number well', 'Check didnt read number well']
IMG_8386grayimage3.jpeg
height 7596
Height 2 7596
IMG_8386grayimage3.jpeg
['10387', '9.09-40.607.07', '12.73', '28.63', '31.08', '11.10', '10.52', '872', '804', '357', '395', '311', '167', '1,525', '1,281', '5', '190', '2/', '0', '0', '-34', '05/03', '05/04', '05/05', '05/06', '05/07', '33']
0.84781

In [9]:
workbook = xlsxwriter.Workbook('5_9_21_Test_Data.xlsx',options={'nan_inf_to_errors': True})
cell_format = workbook.add_format()
ten_day_ma = np.asarray(image_one_data)
inflow_data = np.asarray(inflows)
large_scale_buying_exp = np.asarray(large_scale_buying)
free_float_exp = np.asarray(free_float)
wks = workbook.add_worksheet()


wks.write(0,0,'10 day name')
wks.write(0,1,'10 day MA (USD)')
wks.write(0,2,'inflow/outflow (USD) name')
wks.write(0,3,'inflow')
wks.write(0,4,'outflow')
wks.write(0,5,'large scale buying (USD) name')
wks.write(0,6,'first buying')
wks.write(0,7,'second buying')
wks.write(0,8,'third buying')
wks.write(0,9,'fourth buying')
wks.write(0,10,'fifth buying')
wks.write(0,11,'float name')
wks.write(0,12,'free float in millions')
for i in range(len(inflows)):
    wks.write(i+1,0,ten_day_ma[i,0])
    wks.write(i+1,1,float(ten_day_ma[i,1]))
    wks.write(i+1,2,inflow_data[i,0])
    wks.write(i+1,3,float(inflow_data[i,1]))
    wks.write(i+1,4,float(outflows[i]))
    wks.write(i+1,5,image_threes[i,0])
    for x in range(5):
        arr = np.asarray(large_scale_buying_exp[i])
        try:
            floater = float(arr[x])
            print(arr[x])
            cell_format.set_bg_color('white')
            wks.write(i+1,x+6,floater)
        except ValueError:
            print(arr[x])
            cell_format.set_bg_color('red')
            wks.write(i+1,x+6,arr[x])
        except IndexError:
            wks.write(i+1,x+6, 'Index Error')
    cell_format.set_bg_color('white')
    wks.write(i+1,11,free_float_exp[i,0])
    wks.write(i+1,12,float(free_float_exp[i,1]))
workbook.close()

-207000.0
-443000.0
-40000.0
-147000.0
123000.0
Check didnt read number well
Check didnt read number well
Check didnt read number well
Check didnt read number well
Check didnt read number well
0.0
-240000.0
-265000.0
0.0
221000.0
-4957000.0
-1317000.0
-2495000.0
-135000.0
-3162000.0
0.0
-1881000.0
0.0
214000.0
929000.0
0.0
0.0
-49000.0
-59000.0
107000.0
0.0
-39000.0
0.0
0.0
-21000.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-24000.0
0.0
-35000.0
0.0
0.0
-5000.0
-33000.0
-622000.0
-503000.0
-701000.0
0.0
208000.0
-8000.0
-7000.0
-11000.0
147000.0
375000.0
-2478000.0
-211000.0
0.0
0.0
417000.0
-58000.0
-33000.0
24000.0
68000.0
151000.0
-504000.0
-4701000.0
-393000.0
859000.0
3244000.0
0.0
0.0
0.0
3000.0
749000.0
0.0
70000.0
-11000000.0
2000000.0
5000000.0
0.0
0.0
-566000.0
1386000.0
2636000.0
0.0
0.0
0.0
0.0
1976000.0
-60000.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
coord problem
coord problem
coord problem
coord problem
coord problem
-2631000.0
-3940000.0
-1570000.0
-5136000.0
2527000.0
-767000.0
0.0